# 中醫脈象分類：多數據庫相似度比對系統

**版本 7.0 - 支援「沉、中、浮」三種壓力維度**

**流程概覽:**
1.  **一次性建立多個數據庫**: 程式會自動掃描 `pulse_data` 下的 `沉`, `中`, `浮` 三個子資料夾，並為每一個都建立獨立且永久儲存的「指紋數據庫」。
2.  **執行預測**: 使用者需要提供兩項資訊：
    a. 一個新的未知 CSV 檔案。
    b. 該次測量的壓力級別（`沉`, `中`, 或 `浮`）。
3.  **精準比對**: 系統會根據使用者指定的壓力級別，載入對應的數據庫進行比對，並回傳最相似的結果。

### 步驟 1: 匯入函式庫與定義特徵提取函式

In [1]:
import os
import pandas as pd
import numpy as np
import joblib
import json
from scipy.signal import find_peaks
from scipy.spatial.distance import euclidean
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm

def extract_features(waveform_data):
    """從一個 (n_points, 2) 的波形數據中提取特徵向量。"""
    y_values = waveform_data[:, 1]
    mean_y = np.mean(y_values)
    std_y = np.std(y_values)
    max_y = np.max(y_values)
    min_y = np.min(y_values)
    diff_y = np.diff(y_values, n=1)
    mean_diff = np.mean(diff_y) if diff_y.size > 0 else 0
    std_diff = np.std(diff_y) if diff_y.size > 0 else 0
    peaks, _ = find_peaks(y_values, height=mean_y)
    num_peaks = len(peaks)
    feature_vector = np.array([
        mean_y, std_y, max_y, min_y,
        mean_diff, std_diff, num_peaks
    ])
    return feature_vector

### 步驟 2: (首次執行) 建立並儲存所有壓力級別的數據庫

**注意**：這個儲存格會一次性地為 `沉`, `中`, `浮` 三個子資料夾都建立數據庫。您只需要在第一次執行，或更新了 `pulse_data` 中的檔案後才需要重新執行。

In [2]:
def create_all_databases(base_folder):
    """掃描子資料夾 (沉, 中, 浮) 並為每一個都建立數據庫。"""
    pressure_levels = ['沉', '中', '浮']
    
    for level in pressure_levels:
        folder_path = os.path.join(base_folder, level)
        print(f"\n--- 正在處理壓力級別: '{level}' --- ")
        
        if not os.path.exists(folder_path):
            print(f"警告：找不到資料夾 '{folder_path}'，已跳過。")
            continue
            
        files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]
        if not files:
            print(f"警告：在 '{folder_path}' 中找不到任何 CSV 檔案，已跳過。")
            continue
        
        reference_features = []
        reference_labels = []
        
        for file_name in tqdm(files, desc=f"建立 '{level}' 級數據庫"):
            label = os.path.splitext(file_name)[0]
            file_path = os.path.join(folder_path, file_name)
            try:
                df = pd.read_csv(file_path, header=None, usecols=[0, 1]).apply(pd.to_numeric, errors='coerce').dropna()
                if df.empty: continue
                features = extract_features(df.to_numpy())
                reference_features.append(features)
                reference_labels.append(label)
            except Exception: continue
        
        if not reference_features:
            print(f"警告: 在 '{level}' 資料夾中沒有成功處理任何檔案。")
            continue
            
        reference_features = np.array(reference_features)
        scaler = StandardScaler()
        scaled_reference_features = scaler.fit_transform(reference_features)
        
        # --- 儲存特定級別的物件到檔案 ---
        np.save(f'ref_features_{level}.npy', scaled_reference_features)
        joblib.dump(scaler, f'ref_scaler_{level}.pkl')
        with open(f'ref_labels_{level}.json', 'w', encoding='utf-8') as f:
            json.dump(reference_labels, f, ensure_ascii=False)
        
        print(f"'{level}' 級指紋數據庫已成功建立並儲存！")

# --- 執行建立所有數據庫的流程 ---
DATA_FOLDER = 'pulse_data'
create_all_databases(DATA_FOLDER)


--- 正在處理壓力級別: '沉' --- 


建立 '沉' 級數據庫: 100%|██████████| 4/4 [00:00<00:00, 228.34it/s]


'沉' 級指紋數據庫已成功建立並儲存！

--- 正在處理壓力級別: '中' --- 


建立 '中' 級數據庫: 100%|██████████| 18/18 [00:00<00:00, 306.06it/s]


'中' 級指紋數據庫已成功建立並儲存！

--- 正在處理壓力級別: '浮' --- 


建立 '浮' 級數據庫: 100%|██████████| 7/7 [00:00<00:00, 292.50it/s]

'浮' 級指紋數據庫已成功建立並儲存！


### 步驟 3: (可重複執行) 載入指定數據庫並進行預測

未來您只需要修改 `pressure_level_to_predict` 和 `new_csv_path` 兩個變數，然後執行這個區塊即可。

In [3]:
def load_specific_database(pressure_level):
    """根據指定的壓力級別，從檔案載入對應的指紋數據庫。"""
    try:
        features = np.load(f'ref_features_{pressure_level}.npy')
        scaler = joblib.load(f'ref_scaler_{pressure_level}.pkl')
        with open(f'ref_labels_{pressure_level}.json', 'r', encoding='utf-8') as f:
            labels = json.load(f)
        print(f"'{pressure_level}' 級指紋數據庫載入成功！")
        return features, labels, scaler
    except FileNotFoundError:
        print(f"錯誤：找不到 '{pressure_level}' 級的數據庫檔案。請先執行『步驟 2』來建立數據庫。")
        return None, None, None

def find_most_similar(new_csv_path, reference_features, reference_labels, scaler):
    """比對新的CSV檔案，找出最相似的標準樣本。"""
    try:
        df_new = pd.read_csv(new_csv_path, header=None, usecols=[0, 1]).apply(pd.to_numeric, errors='coerce').dropna()
        if df_new.empty: return "錯誤：新的CSV檔案為空或無法解析。"
        new_features = extract_features(df_new.to_numpy())
        scaled_new_features = scaler.transform(new_features.reshape(1, -1))
        distances = [euclidean(scaled_new_features[0], ref_feature) for ref_feature in reference_features]
        closest_index = np.argmin(distances)
        result = {
            '檔案名稱': os.path.basename(new_csv_path),
            '最相似的標準樣本': reference_labels[closest_index],
            '相似度(距離)': f"{distances[closest_index]:.4f}" # 距離越小代表越相似
        }
        return result
    except Exception as e:
        return f"預測過程中發生錯誤: {e}"

# --- 【新的執行預測流程】 ---

# 1. 使用者互動式輸入壓力級別
valid_levels = ['沉', '中', '浮']
pressure_level_to_predict = '浮'

# 2. 指定您要預測的新檔案路徑 (這裡仍然需要手動修改)
new_csv_path = 'pulse_data/浮/平脈.csv'
# ----------------------------------------------------

print(f"\n準備使用 '{pressure_level_to_predict}' 級數據庫進行預測...")
ref_features, ref_labels, ref_scaler = load_specific_database(pressure_level_to_predict)

if ref_features is not None:
    if os.path.exists(new_csv_path):
        prediction = find_most_similar(new_csv_path, ref_features, ref_labels, ref_scaler)
        print(f"\n對檔案 '{os.path.basename(new_csv_path)}' 的比對結果:")
        if isinstance(prediction, dict):
            for key, value in prediction.items():
                print(f"  - {key}: {value}")
        else:
            print(prediction)
    else:
        print(f"---! 檔案未找到 !---")
        print(f"請將 'new_csv_path' 的值從 '{new_csv_path}' 改為您電腦中真實的CSV檔案路徑!")


準備使用 '浮' 級數據庫進行預測...
'浮' 級指紋數據庫載入成功！

對檔案 '平脈.csv' 的比對結果:
  - 檔案名稱: 平脈.csv
  - 最相似的標準樣本: 平脈
  - 相似度(距離): 0.0000
